In [1]:
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages') # For cv2 finding
import os, glob, math, time
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
import gc

Using TensorFlow backend.


### Load 500 images and their target values from the available datasets 

In [2]:
valid_data = np.load('data/tensors/training/valid_data_224.txt.npy')[0:500]
valid_targets = np.load('data/tensors/training/valid_targets.txt.npy')[0:500]

### Tune the following parameters:
batch_size - [24, 32, 64]

epochs - [3, 5, 10]

optimizer - ['Adam', 'RMSprop']

dropout_rate for two layers added after the FCL added - [0.0, 0.1, 0.2, 0.3]

activation function for the two FCL added - ['relu', 'tanh', 'linear']

neurons - number of nodes in the two TCL added - [1024, 512]

train_after - [84, 78]              



In [3]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K
from keras.layers import Input
from sklearn.model_selection import GridSearchCV

def create_model(dropout_rate, optimizer, activation, neurons, train_after):
    # create the base pre-trained model
    base_model = MobileNetV2(weights='imagenet', include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # # let's add a fully-connected layer
    x = Dense(neurons, activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(neurons, activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    # and a logistic layer 
    predictions = Dense(10, activation='softmax')(x)



    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)


    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])

    
    return model
model = KerasClassifier(build_fn=create_model, dropout_rate=0.2, optimizer='Adam', activation='relu', neurons=1024, train_after=84)



Optimize for Optimizer - Adam or RMSprop

Constant values:

batch_size = 32

epochs = 5

dropout_rate = 0.2

activation = 'relu'

neurons = 1024


In [4]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K
from keras.layers import Input
from sklearn.model_selection import GridSearchCV

# define the grid search parameters
batch_size = [32]
epochs = [5]
optimizer = ['Adam', 'RMSprop']

param_grid = dict(batch_size=batch_size, 
                  epochs=epochs, 
                  optimizer=optimizer)


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, verbose=5)
grid_result = grid.fit(valid_data, valid_targets)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] batch_size=32, epochs=5, optimizer=Adam .........................


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:306: UserWarning: MobileNet shape is undefined. Weights for input shape(224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


9412608/9406464 [==============================] - 1s 0us/step
Epoch 1/5
333/333 [==============================] - 15s 46ms/step - loss: 1.7630 - acc: 0.4384
Epoch 2/5
333/333 [==============================] - 4s 13ms/step - loss: 0.4166 - acc: 0.8919
Epoch 3/5
333/333 [==============================] - 4s 13ms/step - loss: 0.3324 - acc: 0.9219
Epoch 4/5
333/333 [==============================] - 4s 13ms/step - loss: 0.1171 - acc: 0.9489
Epoch 5/5
333/333 [==============================] - 1s 3ms/step
[CV]  batch_size=32, epochs=5, optimizer=Adam, score=0.5628742525677481, total=  52.5s
[CV] batch_size=32, epochs=5, optimizer=Adam .........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   53.5s remaining:    0.0s


Epoch 1/5
333/333 [==============================] - 14s 41ms/step - loss: 1.5776 - acc: 0.5015
Epoch 2/5
333/333 [==============================] - 4s 13ms/step - loss: 0.3145 - acc: 0.9159
Epoch 3/5
333/333 [==============================] - 4s 13ms/step - loss: 0.4946 - acc: 0.8859
Epoch 4/5
333/333 [==============================] - 4s 13ms/step - loss: 0.2010 - acc: 0.9279
Epoch 5/5
333/333 [==============================] - 1s 3ms/step
[CV]  batch_size=32, epochs=5, optimizer=Adam, score=0.407185628920972, total=  49.9s
[CV] batch_size=32, epochs=5, optimizer=Adam .........................


...

Epoch 1/5
333/333 [==============================] - 15s 44ms/step - loss: 3.1133 - acc: 0.1772
Epoch 2/5
333/333 [==============================] - 4s 13ms/step - loss: 1.4517 - acc: 0.5375
Epoch 3/5
333/333 [==============================] - 4s 13ms/step - loss: 0.9251 - acc: 0.7387
Epoch 4/5
333/333 [==============================] - 4s 13ms/step - loss: 0.8202 - acc: 0.7868
Epoch 5/5
333/333 [==============================] - 1s 3ms/step
[CV]  batch_size=32, epochs=5, optimizer=RMSprop, score=0.34131736598328916, total=  53.9s
[CV] batch_size=32, epochs=5, optimizer=RMSprop ......................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  3.6min remaining:    0.0s


Epoch 1/5
333/333 [==============================] - 15s 47ms/step - loss: 3.2839 - acc: 0.1291
Epoch 2/5
333/333 [==============================] - 4s 13ms/step - loss: 1.8948 - acc: 0.3213
Epoch 3/5
333/333 [==============================] - 4s 13ms/step - loss: 1.3111 - acc: 0.5766
Epoch 4/5
333/333 [==============================] - 4s 13ms/step - loss: 0.8661 - acc: 0.7598
Epoch 5/5
333/333 [==============================] - 1s 3ms/step
[CV]  batch_size=32, epochs=5, optimizer=RMSprop, score=0.5508982037712714, total=  57.2s
[CV] batch_size=32, epochs=5, optimizer=RMSprop ......................
Epoch 1/5
334/334 [==============================] - 17s 50ms/step - loss: 3.4092 - acc: 0.1257
Epoch 2/5
334/334 [==============================] - 4s 13ms/step - loss: 1.7782 - acc: 0.4012
Epoch 3/5
334/334 [==============================] - 4s 13ms/step - loss: 1.2738 - acc: 0.5958
Epoch 4/5
334/334 [==============================] - 4s 13ms/step - loss: 0.8795 - acc: 0.7605
Epoch 5/5
33

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  5.6min finished


Epoch 1/5
500/500 [==============================] - 23s 46ms/step - loss: 1.3114 - acc: 0.6020
Epoch 2/5
500/500 [==============================] - 6s 13ms/step - loss: 0.3023 - acc: 0.9180
Epoch 3/5
500/500 [==============================] - 6s 13ms/step - loss: 0.2932 - acc: 0.9200
Epoch 4/5
500/500 [==============================] - 6s 13ms/step - loss: 0.2117 - acc: 0.9580
Epoch 5/5
500/500 [==============================] - 6s 13ms/step - loss: 0.2213 - acc: 0.9360
Best: 0.560000 using {'batch_size': 32, 'epochs': 5, 'optimizer': 'Adam'}
0.560000 (0.123922) with: {'batch_size': 32, 'epochs': 5, 'optimizer': 'Adam'}
0.520000 (0.135356) with: {'batch_size': 32, 'epochs': 5, 'optimizer': 'RMSprop'}


### Optimize FCL layers parameters

Constant values:

Optimizer = Adam

batch_size = 32

activation = 'relu', 'tanh', 'linear'

epochs = 5

dropout_rate = 0.2, 0.3, 0.4

neurons = 512 or 1024


In [4]:
# define the grid search parameters
batch_size = [32]
epochs = [5]
optimizer = ['Adam']
dropout_rate= [ 0.2, 0.3, 0.4]
activation = ['relu', 'tanh', 'linear']
neurons = [512, 1024]

param_grid = dict(batch_size=batch_size, 
                  epochs=epochs, 
                  optimizer=optimizer, 
                  dropout_rate=dropout_rate, 
                  activation=activation,
                  neurons=neurons)


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, verbose=5)
grid_result = grid.fit(valid_data, valid_targets)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] activation=relu, batch_size=32, dropout_rate=0.2, epochs=5, neurons=512, optimizer=Adam 


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:306: UserWarning: MobileNet shape is undefined. Weights for input shape(224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/5
333/333 [==============================] - 14s 43ms/step - loss: 1.6606 - acc: 0.4775
Epoch 2/5
333/333 [==============================] - 4s 13ms/step - loss: 0.3102 - acc: 0.9219
Epoch 3/5
333/333 [==============================] - 4s 13ms/step - loss: 0.2006 - acc: 0.9339
Epoch 4/5
333/333 [==============================] - 4s 13ms/step - loss: 0.2058 - acc: 0.9459
Epoch 5/5
333/333 [==============================] - 1s 3ms/step
[CV]  activation=relu, batch_size=32, dropout_rate=0.2, epochs=5, neurons=512, optimizer=Adam, score=0.5269461081413451, total=  48.5s
[CV] activation=relu, batch_size=32, dropout_rate=0.2, epochs=5, neurons=512, optimizer=Adam 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   49.6s remaining:    0.0s


Epoch 1/5
333/333 [==============================] - 14s 41ms/step - loss: 1.6538 - acc: 0.4474
Epoch 2/5
333/333 [==============================] - 4s 13ms/step - loss: 0.3011 - acc: 0.9009
Epoch 3/5
333/333 [==============================] - 4s 13ms/step - loss: 0.2730 - acc: 0.9459
Epoch 4/5
333/333 [==============================] - 4s 13ms/step - loss: 0.2729 - acc: 0.9309
Epoch 5/5
333/333 [==============================] - 1s 3ms/step
[CV]  activation=relu, batch_size=32, dropout_rate=0.2, epochs=5, neurons=512, optimizer=Adam, score=0.8443113779593371, total=  49.6s
[CV] activation=relu, batch_size=32, dropout_rate=0.2, epochs=5, neurons=512, optimizer=Adam 


...

333/333 [==============================] - 72s 216ms/step - loss: 1.6130 - acc: 0.5075
Epoch 2/5
333/333 [==============================] - 4s 13ms/step - loss: 0.2628 - acc: 0.9099
Epoch 3/5
333/333 [==============================] - 4s 13ms/step - loss: 0.1041 - acc: 0.9760
Epoch 4/5
333/333 [==============================] - 4s 13ms/step - loss: 0.0506 - acc: 0.9880
Epoch 5/5
333/333 [==============================] - 1s 4ms/step
[CV]  activation=tanh, batch_size=32, dropout_rate=0.3, epochs=5, neurons=512, optimizer=Adam, score=0.9041916171233811, total= 3.6min
[CV] activation=tanh, batch_size=32, dropout_rate=0.3, epochs=5, neurons=512, optimizer=Adam 
Epoch 1/5
333/333 [==============================] - 77s 231ms/step - loss: 1.5233 - acc: 0.4775
Epoch 2/5
333/333 [==============================] - 4s 13ms/step - loss: 0.2775 - acc: 0.9129
Epoch 3/5
333/333 [==============================] - 4s 13ms/step - loss: 0.1328 - acc: 0.9520
Epoch 4/5
333/333 [============================

Epoch 1/5
333/333 [==============================] - 118s 355ms/step - loss: 2.0370 - acc: 0.4535
Epoch 2/5
333/333 [==============================] - 4s 13ms/step - loss: 0.2425 - acc: 0.9309
Epoch 3/5
333/333 [==============================] - 4s 13ms/step - loss: 0.0847 - acc: 0.9700
Epoch 4/5
333/333 [==============================] - 4s 13ms/step - loss: 0.1485 - acc: 0.9550
Epoch 5/5
333/333 [==============================] - 1s 4ms/step
[CV]  activation=tanh, batch_size=32, dropout_rate=0.4, epochs=5, neurons=1024, optimizer=Adam, score=0.5748503004719397, total= 5.8min
[CV] activation=tanh, batch_size=32, dropout_rate=0.4, epochs=5, neurons=1024, optimizer=Adam 
Epoch 1/5
334/334 [==============================] - 123s 368ms/step - loss: 2.2923 - acc: 0.3623
Epoch 2/5
334/334 [==============================] - 5s 14ms/step - loss: 0.2798 - acc: 0.9072
Epoch 3/5
334/334 [==============================] - 5s 14ms/step - loss: 0.2557 - acc: 0.9222
Epoch 4/5
334/334 [==============

Epoch 1/5
334/334 [==============================] - 172s 516ms/step - loss: 2.2229 - acc: 0.3772
Epoch 2/5
334/334 [==============================] - 5s 14ms/step - loss: 0.3024 - acc: 0.9012
Epoch 3/5
334/334 [==============================] - 5s 14ms/step - loss: 0.1443 - acc: 0.9581
Epoch 4/5
334/334 [==============================] - 5s 14ms/step - loss: 0.1612 - acc: 0.9431
Epoch 5/5
334/334 [==============================] - 1s 4ms/step
[CV]  activation=linear, batch_size=32, dropout_rate=0.3, epochs=5, neurons=512, optimizer=Adam, score=0.8734939759036144, total= 8.1min
[CV] activation=linear, batch_size=32, dropout_rate=0.3, epochs=5, neurons=1024, optimizer=Adam 
Epoch 1/5
333/333 [==============================] - 181s 543ms/step - loss: 2.7409 - acc: 0.3754
Epoch 2/5
333/333 [==============================] - 5s 14ms/step - loss: 0.4963 - acc: 0.8589
Epoch 3/5
333/333 [==============================] - 5s 14ms/step - loss: 0.1618 - acc: 0.9550
Epoch 4/5
333/333 [===========

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed: 254.8min finished


Epoch 1/5
500/500 [==============================] - 239s 479ms/step - loss: 1.4203 - acc: 0.5420
Epoch 2/5
500/500 [==============================] - 7s 14ms/step - loss: 0.2161 - acc: 0.9460
Epoch 3/5
500/500 [==============================] - 7s 14ms/step - loss: 0.0918 - acc: 0.9760
Epoch 4/5
500/500 [==============================] - 7s 14ms/step - loss: 0.0650 - acc: 0.9800
Epoch 5/5
500/500 [==============================] - 7s 14ms/step - loss: 0.0697 - acc: 0.9760
Best: 0.932000 using {'activation': 'tanh', 'batch_size': 32, 'dropout_rate': 0.4, 'epochs': 5, 'neurons': 512, 'optimizer': 'Adam'}
0.698000 (0.130875) with: {'activation': 'relu', 'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 5, 'neurons': 512, 'optimizer': 'Adam'}
0.568000 (0.180036) with: {'activation': 'relu', 'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 5, 'neurons': 1024, 'optimizer': 'Adam'}
0.694000 (0.119659) with: {'activation': 'relu', 'batch_size': 32, 'dropout_rate': 0.3, 'epochs': 5, 'neurons': 

### Tune the learning rate and batch size
    
batch_size - [24, 32, 64]

epochs - [7, 15]

optimizer - ['Adam']

dropout_rate for two layers added after the FCL added - [0.4]

activation function for the two FCL added - ['tanh']

neurons - number of nodes in the two TCL added - [512]


In [3]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K
from keras.layers import Input
from sklearn.model_selection import GridSearchCV
from keras.optimizers import Adam


def create_model(learn_rate=0.01, alpha=1.0, depth_multiplier=1.0):
    # create the base pre-trained model
    base_model = MobileNetV2(weights='imagenet', include_top=False, alpha=alpha, depth_multiplier=depth_multiplier)

    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # # let's add a fully-connected layer
    x = Dense(512, activation='tanh')(x)
    x = Dropout(0.4)(x)
    x = Dense(512, activation='tanh')(x)
    x = Dropout(0.4)(x)
    # and a logistic layer 
    predictions = Dense(10, activation='softmax')(x)



    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    optim = Adam(lr=learn_rate)
    model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=["accuracy"])

    
    return model
model = KerasClassifier(build_fn=create_model, learn_rate=0.01, alpha=1.0, depth_multiplier=1.0)





In [4]:
# define the grid search parameters
epochs = [7, 15]
batch_size = [16, 32, 64]
learn_rate= [0.0001, 0.001]
param_grid = dict(batch_size=batch_size, 
                  epochs=epochs, 
                  learn_rate=learn_rate)


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, verbose=5)
grid_result = grid.fit(valid_data, valid_targets)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] batch_size=16, epochs=7, learn_rate=0.0001 ......................


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:306: UserWarning: MobileNet shape is undefined. Weights for input shape(224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/7
333/333 [==============================] - 14s 43ms/step - loss: 2.2120 - acc: 0.2583
Epoch 2/7
333/333 [==============================] - 5s 14ms/step - loss: 1.0164 - acc: 0.7718
Epoch 3/7
333/333 [==============================] - 5s 14ms/step - loss: 0.4802 - acc: 0.9249
Epoch 4/7
333/333 [==============================] - 5s 14ms/step - loss: 0.2023 - acc: 0.9880
Epoch 5/7
333/333 [==============================] - 5s 14ms/step - loss: 0.0948 - acc: 0.9970
Epoch 6/7
333/333 [==============================] - 5s 14ms/step - loss: 0.0560 - acc: 1.0000
Epoch 7/7
333/333 [==============================] - 1s 3ms/step
[CV]  batch_size=16, epochs=7, learn_rate=0.0001, score=0.9760479045485309, total= 1.0min
[CV] batch_size=16, epochs=7, learn_rate=0.0001 ......................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.0min remaining:    0.0s


Epoch 1/7
333/333 [==============================] - 14s 42ms/step - loss: 2.0537 - acc: 0.3123
Epoch 2/7
333/333 [==============================] - 5s 14ms/step - loss: 0.8984 - acc: 0.7658
Epoch 3/7
333/333 [==============================] - 5s 14ms/step - loss: 0.3878 - acc: 0.9399
Epoch 4/7
333/333 [==============================] - 5s 14ms/step - loss: 0.1784 - acc: 0.9850
Epoch 5/7
333/333 [==============================] - 5s 14ms/step - loss: 0.1036 - acc: 1.0000
Epoch 6/7
333/333 [==============================] - 5s 14ms/step - loss: 0.0595 - acc: 0.9970
Epoch 7/7
333/333 [==============================] - 1s 3ms/step
[CV]  batch_size=16, epochs=7, learn_rate=0.0001, score=0.9580838323353293, total= 1.0min
[CV] batch_size=16, epochs=7, learn_rate=0.0001 ......................


...

333/333 [==============================] - 5s 15ms/step - loss: 0.1919 - acc: 0.9429
Epoch 14/15
333/333 [==============================] - 5s 15ms/step - loss: 0.1846 - acc: 0.9339
Epoch 15/15
333/333 [==============================] - 1s 4ms/step
[CV]  batch_size=16, epochs=15, learn_rate=0.001, score=0.5748502997581116, total= 2.4min
[CV] batch_size=16, epochs=15, learn_rate=0.001 ......................
Epoch 1/15
333/333 [==============================] - 30s 91ms/step - loss: 1.7348 - acc: 0.4715
Epoch 2/15
333/333 [==============================] - 5s 14ms/step - loss: 0.4997 - acc: 0.8348
Epoch 3/15
333/333 [==============================] - 5s 15ms/step - loss: 0.2418 - acc: 0.9099
Epoch 4/15
333/333 [==============================] - 5s 15ms/step - loss: 0.2097 - acc: 0.9399
Epoch 5/15
333/333 [==============================] - 5s 15ms/step - loss: 0.3093 - acc: 0.9039
Epoch 6/15
333/333 [==============================] - 5s 15ms/step - loss: 0.3331 - acc: 0.9219
Epoch 7/15
33

333/333 [==============================] - 1s 4ms/step
[CV]  batch_size=32, epochs=15, learn_rate=0.001, score=0.6107784434706865, total= 3.8min
[CV] batch_size=32, epochs=15, learn_rate=0.001 ......................
Epoch 1/15
333/333 [==============================] - 65s 194ms/step - loss: 1.6528 - acc: 0.4474
Epoch 2/15
333/333 [==============================] - 4s 13ms/step - loss: 0.4587 - acc: 0.8739
Epoch 3/15
333/333 [==============================] - 4s 13ms/step - loss: 0.1881 - acc: 0.9399
Epoch 4/15
333/333 [==============================] - 4s 13ms/step - loss: 0.1110 - acc: 0.9640
Epoch 5/15
333/333 [==============================] - 4s 13ms/step - loss: 0.1039 - acc: 0.9700
Epoch 6/15
333/333 [==============================] - 4s 13ms/step - loss: 0.1046 - acc: 0.9640
Epoch 7/15
333/333 [==============================] - 4s 13ms/step - loss: 0.3035 - acc: 0.9309
Epoch 8/15
333/333 [==============================] - 4s 13ms/step - loss: 0.2505 - acc: 0.9219
Epoch 9/15
333

333/333 [==============================] - 4s 13ms/step - loss: 0.5119 - acc: 0.8559
Epoch 3/15
333/333 [==============================] - 4s 13ms/step - loss: 0.1291 - acc: 0.9700
Epoch 4/15
333/333 [==============================] - 4s 13ms/step - loss: 0.0547 - acc: 0.9880
Epoch 5/15
333/333 [==============================] - 4s 13ms/step - loss: 0.0433 - acc: 0.9850
Epoch 6/15
333/333 [==============================] - 4s 13ms/step - loss: 0.0849 - acc: 0.9790
Epoch 7/15
333/333 [==============================] - 4s 13ms/step - loss: 0.0502 - acc: 0.9820
Epoch 8/15
333/333 [==============================] - 4s 13ms/step - loss: 0.0867 - acc: 0.9700
Epoch 9/15
333/333 [==============================] - 4s 13ms/step - loss: 0.0491 - acc: 0.9850
Epoch 10/15
333/333 [==============================] - 4s 13ms/step - loss: 0.0623 - acc: 0.9850
Epoch 11/15
333/333 [==============================] - 4s 13ms/step - loss: 0.0633 - acc: 0.9760
Epoch 12/15
333/333 [============================

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 120.2min finished


Epoch 1/7
500/500 [==============================] - 131s 262ms/step - loss: 1.9178 - acc: 0.3660
Epoch 2/7
500/500 [==============================] - 8s 16ms/step - loss: 0.6937 - acc: 0.8620
Epoch 3/7
500/500 [==============================] - 8s 16ms/step - loss: 0.2539 - acc: 0.9640
Epoch 4/7
500/500 [==============================] - 8s 16ms/step - loss: 0.0925 - acc: 1.0000
Epoch 5/7
500/500 [==============================] - 8s 16ms/step - loss: 0.0544 - acc: 1.0000
Epoch 6/7
500/500 [==============================] - 8s 16ms/step - loss: 0.0374 - acc: 0.9960
Epoch 7/7
500/500 [==============================] - 8s 16ms/step - loss: 0.0282 - acc: 1.0000
Best: 0.970000 using {'batch_size': 16, 'epochs': 7, 'learn_rate': 0.0001}
0.970000 (0.008439) with: {'batch_size': 16, 'epochs': 7, 'learn_rate': 0.0001}
0.502000 (0.096830) with: {'batch_size': 16, 'epochs': 7, 'learn_rate': 0.001}
0.966000 (0.010163) with: {'batch_size': 16, 'epochs': 15, 'learn_rate': 0.0001}
0.632000 (0.04363

### Tune the following parameters:

alpha = [0.35, 0.50, 0.75, 1.0, 1.3, 1.4]


In [8]:
# define the grid search parameters
epochs = [7]
batch_size = [16]
learn_rate= [0.0001]
alpha = [0.35, 0.50, 0.75, 1.0, 1.3, 1.4]
depth_multiplier = [1.0]
param_grid = dict(batch_size=batch_size, 
                  epochs=epochs, 
                  learn_rate=learn_rate,
                  alpha = alpha,
                  depth_multiplier = depth_multiplier)


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, verbose=5)
grid_result = grid.fit(valid_data, valid_targets)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] alpha=0.35, batch_size=16, depth_multiplier=1.0, epochs=7, learn_rate=0.0001 


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:306: UserWarning: MobileNet shape is undefined. Weights for input shape(224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/7
333/333 [==============================] - 17s 50ms/step - loss: 2.3263 - acc: 0.1862
Epoch 2/7
333/333 [==============================] - 3s 9ms/step - loss: 1.5783 - acc: 0.4955
Epoch 3/7
333/333 [==============================] - 3s 9ms/step - loss: 1.0433 - acc: 0.7598
Epoch 4/7
333/333 [==============================] - 3s 9ms/step - loss: 0.6272 - acc: 0.8829
Epoch 5/7
333/333 [==============================] - 3s 9ms/step - loss: 0.3767 - acc: 0.9580
Epoch 6/7
333/333 [==============================] - 3s 9ms/step - loss: 0.2428 - acc: 0.9730
Epoch 7/7
333/333 [==============================] - 1s 2ms/step
[CV]  alpha=0.35, batch_size=16, depth_multiplier=1.0, epochs=7, learn_rate=0.0001, score=0.9760479041916168, total= 1.0min
[CV] alpha=0.35, batch_size=16, depth_multiplier=1.0, epochs=7, learn_rate=0.0001 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.0min remaining:    0.0s


Epoch 1/7
333/333 [==============================] - 18s 55ms/step - loss: 2.3103 - acc: 0.2132
Epoch 2/7
333/333 [==============================] - 3s 9ms/step - loss: 1.4476 - acc: 0.5946
Epoch 3/7
333/333 [==============================] - 3s 9ms/step - loss: 0.8961 - acc: 0.7598
Epoch 4/7
333/333 [==============================] - 3s 9ms/step - loss: 0.4999 - acc: 0.9039
Epoch 5/7
333/333 [==============================] - 3s 9ms/step - loss: 0.3069 - acc: 0.9730
Epoch 6/7
333/333 [==============================] - 3s 9ms/step - loss: 0.2312 - acc: 0.9880
Epoch 7/7
333/333 [==============================] - 1s 2ms/step
[CV]  alpha=0.35, batch_size=16, depth_multiplier=1.0, epochs=7, learn_rate=0.0001, score=0.9161676646706587, total= 1.1min
[CV] alpha=0.35, batch_size=16, depth_multiplier=1.0, epochs=7, learn_rate=0.0001 


...

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  4.7min remaining:    0.0s


Epoch 1/7
333/333 [==============================] - 25s 74ms/step - loss: 2.1685 - acc: 0.2703
Epoch 2/7
333/333 [==============================] - 3s 10ms/step - loss: 1.2264 - acc: 0.7027
Epoch 3/7
333/333 [==============================] - 3s 10ms/step - loss: 0.6691 - acc: 0.8589
Epoch 4/7
333/333 [==============================] - 3s 10ms/step - loss: 0.3798 - acc: 0.9429
Epoch 5/7
333/333 [==============================] - 3s 10ms/step - loss: 0.2203 - acc: 0.9910
Epoch 6/7
333/333 [==============================] - 3s 10ms/step - loss: 0.1219 - acc: 1.0000
Epoch 7/7
333/333 [==============================] - 1s 3ms/step
[CV]  alpha=0.5, batch_size=16, depth_multiplier=1.0, epochs=7, learn_rate=0.0001, score=0.9580838323353293, total= 1.4min
[CV] alpha=0.5, batch_size=16, depth_multiplier=1.0, epochs=7, learn_rate=0.0001 
Epoch 1/7
334/334 [==============================] - 26s 77ms/step - loss: 2.2950 - acc: 0.2036
Epoch 2/7
334/334 [==============================] - 3s 10ms/st

15441920/15441408 [==============================] - 1s 0us/step
Epoch 1/7
333/333 [==============================] - 49s 146ms/step - loss: 2.1609 - acc: 0.2703
Epoch 2/7
333/333 [==============================] - 6s 19ms/step - loss: 0.9269 - acc: 0.7688
Epoch 3/7
333/333 [==============================] - 6s 19ms/step - loss: 0.3211 - acc: 0.9520
Epoch 4/7
333/333 [==============================] - 6s 19ms/step - loss: 0.1460 - acc: 0.9970
Epoch 5/7
333/333 [==============================] - 6s 19ms/step - loss: 0.0650 - acc: 1.0000
Epoch 6/7
333/333 [==============================] - 6s 19ms/step - loss: 0.0341 - acc: 1.0000
Epoch 7/7
333/333 [==============================] - 2s 6ms/step
[CV]  alpha=1.3, batch_size=16, depth_multiplier=1.0, epochs=7, learn_rate=0.0001, score=0.9760479041916168, total= 2.8min
[CV] alpha=1.3, batch_size=16, depth_multiplier=1.0, epochs=7, learn_rate=0.0001 
Epoch 1/7
333/333 [==============================] - 52s 157ms/step - loss: 2.0397 - acc: 0.3

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 39.8min finished


Epoch 1/7
500/500 [==============================] - 71s 142ms/step - loss: 2.0606 - acc: 0.3000
Epoch 2/7
500/500 [==============================] - 7s 15ms/step - loss: 0.8433 - acc: 0.8000
Epoch 3/7
500/500 [==============================] - 7s 15ms/step - loss: 0.3422 - acc: 0.9420
Epoch 4/7
500/500 [==============================] - 7s 15ms/step - loss: 0.1385 - acc: 0.9840
Epoch 5/7
500/500 [==============================] - 7s 15ms/step - loss: 0.0803 - acc: 0.9940
Epoch 6/7
500/500 [==============================] - 7s 15ms/step - loss: 0.0523 - acc: 1.0000
Epoch 7/7
500/500 [==============================] - 7s 15ms/step - loss: 0.0306 - acc: 1.0000
Best: 0.960000 using {'alpha': 0.75, 'batch_size': 16, 'depth_multiplier': 1.0, 'epochs': 7, 'learn_rate': 0.0001}
0.948000 (0.024617) with: {'alpha': 0.35, 'batch_size': 16, 'depth_multiplier': 1.0, 'epochs': 7, 'learn_rate': 0.0001}
0.956000 (0.017328) with: {'alpha': 0.5, 'batch_size': 16, 'depth_multiplier': 1.0, 'epochs': 7, '